# GeoNames API to get City and Country information

- [get-started-geo-names-api](https://www.geonames.org/export/web-services.html)
- [enable-webservice](https://www.geonames.org/enablefreewebservice)
- [geonames-search-api](https://www.geonames.org/export/geonames-search.html)
- [download-geo-names-datasets](https://download.geonames.org/export/dump/)

In [41]:
import requests
import pandas as pd
from zipfile import ZipFile
from io import BytesIO

In [42]:
# Download and unzip the Geonames cities file
geonames_url = "http://download.geonames.org/export/dump/cities15000.zip"
response = requests.get(geonames_url)
zip_file = ZipFile(BytesIO(response.content))
file_name = zip_file.namelist()[0]
cities_data = pd.read_csv(zip_file.open(file_name), sep='\t', header=None, names=[
    'geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 'longitude',
    'feature class', 'feature code', 'country_code', 'cc2', 'admin1 code',
    'admin2 code', 'admin3 code', 'admin4 code', 'population', 'elevation',
    'dem', 'timezone', 'modification date'
])

In [57]:
cities_data

,name,alternatenames,latitude,longitude,country_code,country
0,les Escaldes,"Ehskal'des-Ehndzhordani,Escaldes,Escaldes-Engo...",42.50729,1.53414,AD,Andorra
1,Andorra la Vella,"ALV,Ando-la-Vyey,Andora,Andora la Vela,Andora ...",42.50779,1.52109,AD,Andorra
2,Warisan,"Warisan,Warsan,Warīsān,wrsan,ورسان",25.16744,55.40708,AE,United Arab Emirates
3,Umm Al Quwain City,"Oumm al Qaiwain,Oumm al Qaïwaïn,Um al Kawain,U...",25.56473,55.55517,AE,United Arab Emirates
4,Ras Al Khaimah City,"Julfa,Khaimah,RAK City,RKT,Ra's al Khaymah,Ra'...",25.78953,55.94320,AE,United Arab Emirates
...,...,...,...,...,...,...
28435,Bulawayo,"BUQ,Bulavajas,Bulavajo,Bulavejo,Bulawayo,bu la...",-20.15000,28.58333,ZW,Zimbabwe
28436,Bindura,"Bindura,Bindura Town,Kimberley Reefs,Биндура",-17.30192,31.33056,ZW,Zimbabwe
28437,Beitbridge,"Bajtbridz,Bajtbridzh,Beitbridge,Beitbridzas,Be...",-22.21667,30.00000,ZW,Zimbabwe
28438,Epworth,Epworth,-17.89000,31.14750,ZW,Zimbabwe


In [44]:
# Extract city names
cities = cities_data[['asciiname', 'alternatenames', 'country_code']]

In [45]:
# Geonames API settings
username = 'alexon99'  # Remplacez par votre nom d'utilisateur Geonames
base_url = "http://api.geonames.org/searchJSON"

# Function to get city data from Geonames
def get_city_data(city_name, country_code=None):
    params = {
        'q': city_name,
        'maxRows': 1,
        'username': username
    }
    if country_code:
        params['country'] = country_code
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if 'geonames' in data and len(data['geonames']) > 0:
            city_info = data['geonames'][0]
            return {
                'name': city_info.get('name'),
                'country': city_info.get('countryName'),
                'country_code': city_info.get('countryCode'),
                'latitude': city_info.get('lat'),
                'longitude': city_info.get('lng'),
            }
    return None

In [46]:
# Example datasets for beach, family, ski, and golf
beach_cities = ['Los Angeles', 'Miami', 'Barcelona']  # Simplified example
family_friendly_cities = ['Paris', 'New York', 'Tokyo']  # Simplified example
ski_resorts = ['Chamonix', 'Aspen', 'Whistler']  # Simplified example
golf_courses = ['St Andrews', 'Augusta', 'Pebble Beach']  # Simplified example

# Functions to check for beach, family, ski, and golf
def has_beach(city_name):
    return city_name in beach_cities

def is_family_friendly(city_name):
    return city_name in family_friendly_cities

def has_ski_resorts(city_name):
    return city_name in ski_resorts

def has_golf_courses(city_name):
    return city_name in golf_courses

In [48]:
# Get data for all cities and add additional information
city_data = []
for index, row in cities.iterrows():
    asciiname = row['asciiname']
    alternatenames = row['alternatenames']
    country_code = row['country_code']
    data = get_city_data(asciiname, country_code=country_code)
    if data:
        data['alternatenames'] = alternatenames
        data['beach'] = has_beach(data['name'])
        data['family'] = is_family_friendly(data['name'])
        data['ski'] = has_ski_resorts(data['name'])
        data['golf'] = has_golf_courses(data['name'])
        city_data.append(data)

# Create a DataFrame
df = pd.DataFrame(city_data)

KeyboardInterrupt: 

In [5]:
df

,name,country,latitude,longitude,beach,family,ski,golf
0,Paris,France,48.85341,2.3488,False,False,False,False
1,New York,United States,40.71427,-74.00597,False,False,False,False
2,Tokyo,Japan,35.6895,139.69171,False,False,False,False
3,London,United Kingdom,51.50853,-0.12574,False,False,False,False
4,Berlin,Germany,52.52437,13.41053,False,False,False,False


In [ ]:
# Save the DataFrame to a CSV file
df.to_csv("cities_data.csv", index=False)